In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [56]:
import pandas as pd

mclp = pd.read_csv("/content/drive/MyDrive/WCRC2023/센서위치선정.csv", encoding="CP949")
df = pd.read_csv("/content/drive/MyDrive/WCRC2023/데이터/최종 데이터/최종데이터_final_ver1.csv", encoding="CP949")

In [57]:
from sklearn.preprocessing import StandardScaler

data = df.drop(["cctvid"], axis=1)

scaler = StandardScaler()
scaled_feature = scaler.fit_transform(data)
scaled_df = pd.DataFrame(scaled_feature, columns=data.columns)
scaled_df["SUM"] = scaled_df["관측소_거리"] + scaled_df["버스정류장_이용객수"] + scaled_df["지하철역_지표"] + scaled_df["학교_거리"] + scaled_df["처리시설_개수"] + scaled_df["CAI"]
scaled_df["cctvid"] = df["cctvid"].astype(int)

mclp = mclp[["가까운_cctvid", "소재지지번주소"]]
mclp = mclp.rename(columns={"가까운_cctvid":"cctvid"})
mclp = mclp.drop_duplicates(["cctvid"], keep="first")
df = mclp.merge(scaled_df, on=["cctvid"])

df["소음"] = df["지하철역_지표"] + df["버스정류장_이용객수"]
df["대기오염"] = df["처리시설_개수"] + df["CAI"]
df = df.drop(["버스정류장_이용객수", "지하철역_지표", "처리시설_개수", "CAI"], axis=1)

In [58]:
# 입지선정
df.sort_values(by=["관측소_거리", "SUM"], ascending=False).head(3)

,cctvid,소재지지번주소,관측소_거리,학교_거리,SUM,소음,대기오염
190,22482,경기도 양평군 양평읍 양근리 638,2.005977,-0.527548,1.937508,1.008053,-0.548974
29,9589,경기도 성남시 분당구 서현2동 302번지 서당어린이공원(분당34호) 서당초등학교 주변,2.005977,0.316963,1.766248,1.820893,-2.377585
292,22660,경기도 양평군 옥천면 옥천리 784-10 대,2.005977,0.900588,1.319813,-1.613886,0.027133


In [61]:
# 소음 (지하철역_지표 + 버스정류장_이용객수)
df.sort_values(by=["소음"], ascending=False).head(3)

,cctvid,소재지지번주소,관측소_거리,학교_거리,SUM,소음,대기오염
295,2426,경기도 광명시 하안1동 702-3 가림어린이공원,-1.212854,2.066705,4.486645,4.941076,-1.308282
216,19497,경기도 안양시 동안구 비산3동 1057,-1.394543,-0.238917,1.900581,3.710641,-0.176601
23,24517,경기도 오산시 오산동 920-1번지,-1.074070,0.125197,2.690091,3.461575,0.177388


In [62]:
# 건강취약계층
df.sort_values(by=["학교_거리"], ascending=False).head(3)

,cctvid,소재지지번주소,관측소_거리,학교_거리,SUM,소음,대기오염
131,15336,경기도 시흥시 조남동 조남동 171-53,-0.790607,2.996442,7.479628,0.711004,4.562789
183,15520,경기도 안산시 단원구 고잔동 672,-0.328459,2.919337,4.588786,2.293659,-0.295751
248,28693,경기도 이천시 모가면 진가리 157-1번지,1.628291,2.918184,3.376441,-1.631525,0.461491


In [63]:
# 대기오염 (처리시설_개수 + CAI)
df.sort_values(by=["대기오염"], ascending=False).head(3)

,cctvid,소재지지번주소,관측소_거리,학교_거리,SUM,소음,대기오염
131,15336,경기도 시흥시 조남동 조남동 171-53,-0.790607,2.996442,7.479628,0.711004,4.562789
364,7145,경기도 동두천시 상패동 309번지,0.835852,0.172074,4.561049,-0.451712,4.004836
409,36054,경기도 화성시 비봉면 양노리 296-5,-0.056107,-0.330968,2.255434,-1.037685,3.680193
